In [1]:
from datascience import *
import numpy as np
import math
import scipy.stats as stats

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd

# Problem 1. 
An analysis of fat content for a sample of hamburgers can be found in table `hamburger.csv`. We would like to test if fat contents follow a normal distribution using chi-square goodness-of-fit test. We saw how to do goodness-of-fit test for a Poisson Distribution. The following parts will guide you through how to conduct such test for Normal Distribution.

In [2]:
fat = Table.read_table('hamburger.csv')
fat

lb,ub,"numberOfObs in [lb,ub)"
26,28,7
28,30,22
30,32,36
32,34,45
34,36,33
36,38,28
38,40,4


##### 1. What are the Null and Alternative Hypothesis?

* $H_0$: the fat contents follow a normal distribution
* $H_1$: the fat contents do not follow a normal distribution


##### 2. What parameters need to estimated from the data?

mean_u:${\bar{x}}$

variance: s**2

##### 3. What are the estimates of these parameters? Use the class mid-points of 27, 29, ... 39 for your calculation.

In [3]:
fat.column('numberOfObs in [lb,ub)').sum()


175

In [4]:
n =175
mean_u = (27*7+ 29*22+31*36+33*45+35*33+37*28+39*4)/175
mean_u

33.0

In [5]:
variance = ((33-27)**2*7+(33-29)**2*22+(33-31)**2*36+(35-33)**2*33+(37-33)**2*28+(39-33)**2*4)/175
variance

8.411428571428571

##### 4. What are the expected frequencies (expected probability) for each categories of the data? Add this as a new column to the table.

This might be helpful: Let $X$ be a normal distribution with mean $\mu$ and variance $\sigma^2$. Then, the probability $\mathbb{P}(a\le X\le b)$ can be computed using

* stats.norm.cdf(b, $\mu$, $\sigma$) - stats.norm.cdf(a, $\mu$, $\sigma$) 

In [6]:
std = np.sqrt(variance)

In [7]:
std

2.9002462949598904

In [8]:
fat = fat.with_column('expected frequency', stats.norm.cdf(fat.column('ub'),mean_u, std )- stats.norm.cdf(fat.column('lb'),mean_u, std ))

In [9]:
fat

lb,ub,"numberOfObs in [lb,ub)",expected frequency
26,28,7,0.0344565
28,30,22,0.108121
30,32,36,0.214647
32,34,45,0.269754
34,36,33,0.214647
36,38,28,0.108121
38,40,4,0.0344565


##### 5. For complete analysis, add to the table two additional categories: 
* $(-\infty, 26)$ 
* $[40, \infty)$.

In [10]:
a = 1-stats.norm.cdf(40, mean_u, std)
fat = fat.with_row(make_array(40, math.inf, 0 ,a))
fat


lb,ub,"numberOfObs in [lb,ub)",expected frequency
26,28,7,0.0344565
28,30,22,0.108121
30,32,36,0.214647
32,34,45,0.269754
34,36,33,0.214647
36,38,28,0.108121
38,40,4,0.0344565
40,inf,0,0.00789815


In [11]:
b = stats.norm.cdf(26, mean_u, std)-0
fat = fat.with_row(make_array( -math.inf,26, 0 ,b))
fat

lb,ub,"numberOfObs in [lb,ub)",expected frequency
26,28,7,0.0344565
28,30,22,0.108121
30,32,36,0.214647
32,34,45,0.269754
34,36,33,0.214647
36,38,28,0.108121
38,40,4,0.0344565
40,inf,0,0.00789815
-inf,26,0,0.00789815


In [12]:
fat = fat.sort('ub')
fat

lb,ub,"numberOfObs in [lb,ub)",expected frequency
-inf,26,0,0.00789815
26,28,7,0.0344565
28,30,22,0.108121
30,32,36,0.214647
32,34,45,0.269754
34,36,33,0.214647
36,38,28,0.108121
38,40,4,0.0344565
40,inf,0,0.00789815


##### 6. What are the expected counts for each category of the data? Add this as a new column to the table.

In [13]:
fat = fat.with_column('expected', fat.column('expected frequency')*175)
fat

lb,ub,"numberOfObs in [lb,ub)",expected frequency,expected
-inf,26,0,0.00789815,1.38218
26,28,7,0.0344565,6.02989
28,30,22,0.108121,18.9212
30,32,36,0.214647,37.5633
32,34,45,0.269754,47.2069
34,36,33,0.214647,37.5633
36,38,28,0.108121,18.9212
38,40,4,0.0344565,6.02989
40,inf,0,0.00789815,1.38218


##### 7. Clean the table so that it only has four columns:
* lb,
* ub,
* expected number,
* observed number. 

If the expected number of oberservation is less than 5 for one category, combine it with the category adjacent to it.

In [14]:
fat = fat.select('lb','ub','numberOfObs in [lb,ub)', 'expected')
fat

lb,ub,"numberOfObs in [lb,ub)",expected
-inf,26,0,1.38218
26,28,7,6.02989
28,30,22,18.9212
30,32,36,37.5633
32,34,45,47.2069
34,36,33,37.5633
36,38,28,18.9212
38,40,4,6.02989
40,inf,0,1.38218


In [15]:
fat = fat.with_row(make_array(38,math.inf,4,fat.row(7)[3]+fat.row(8)[3]))
fat = fat.with_row(make_array(-math.inf,28,7,fat.row(0)[3]+fat.row(1)[3]))
fat =fat.take([2,3,4,5,6,9,10]).sort('ub')
fat

lb,ub,"numberOfObs in [lb,ub)",expected
-inf,28,7,7.41207
28,30,22,18.9212
30,32,36,37.5633
32,34,45,47.2069
34,36,33,37.5633
36,38,28,18.9212
38,inf,4,7.41207


##### 8. Compute the test statistic.

In [16]:
expected = fat.column('expected')
observed = fat.column('numberOfObs in [lb,ub)')
Q = sum((expected-observed)**2/expected)
Q

7.17339614191457

##### 9. What is the degree of freedom?

In [17]:
df = fat.num_rows-1-2
df

4

##### 10. What is the p-value and what is your conclusion?

In [18]:
pval = stats.chi2.sf(Q,df)
pval

0.1270038751143096

Since p-value is bigger than 0.05, we have enough evidence to accept the null hypothesis. The fat contents follow a normal distribution. 

# Problem 2. 

A sample of adults in Eastern and Central Newfoundland was conducted early in 1988 to examine public attitudes toward government cuts in social spending. Some of the results from this study are described in Morris Saldov, “Public Attitudes to Social Spending in Newfoundland,” Canadian Review of Social Policy, 26, November 1990, pages 10-14. The data is represented in the table below:

<img width=400px src='welfare.png'>

Concerning this data, the
author comments,

* "Respondents who knew someone on social assistance, were more likely to feel that welfare rates were too low, ... "

##### 1. Can we conduct a test of independence here, or is it that we can only carry out a test of homogeneity?

we conduct a test of independence here.

##### 2. Depending on your answer for 1., what are the null and alternative hypotheses?

The null hypothesis and alternative hypothesis are:
* $H_0$: respondents who knew someone on social assistance and not are equally likely to feel that welfare rates were too low
* $H_1$: respomdents who knew someone on social assistance are more likely to feel that welfare rates were too low


##### 3. Compute the test statistic.

In [19]:
observed = np.array([[40, 6], [16, 13], [9,7]])
observed


array([[40,  6],
       [16, 13],
       [ 9,  7]])

In [20]:
observed_margin = np.array([[40,6,46],[16,13,29],[9,7,16],[65,26,91]])
observed_margin

array([[40,  6, 46],
       [16, 13, 29],
       [ 9,  7, 16],
       [65, 26, 91]])

In [21]:
nrows = 3
ncols = 2
expected = np.zeros((nrows,ncols))   #initialization
for i in np.arange(nrows):
    for j in np.arange(ncols):
        expected[i,j] = observed_margin[i,ncols]*observed_margin[nrows,j]/observed_margin[nrows,ncols]
expected

array([[32.85714286, 13.14285714],
       [20.71428571,  8.28571429],
       [11.42857143,  4.57142857]])

In [22]:
Q = ((expected-observed)**2/expected).sum()
Q

10.996205022488756

##### 4. What is the degree of freedom?

In [23]:
df = (nrows-1)*(ncols-1)
df

2

##### 5. What is the p-value and what is your conclusion?

In [24]:
pval = stats.chi2.sf(Q,df)

In [25]:
pval

0.004094533403106778

Since p-vale < 0.05, we reject the null hypothesis. Respomdents who knew someone on social assistance are more likely to feel that welfare rates were too low.
